In [1]:
import json

import requests

from folium import Map, TileLayer, GeoJson
from rasterio.features import bounds as featureBounds

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [6]:
endpoint = "http://0.0.0.0:8081"
collection = "joplin"

In [7]:
with open("index.geojson", "r") as f:
    geojson = json.load(f)

bounds = featureBounds(geojson)
print(bounds)
m = Map(
    tiles="OpenStreetMap",
    location=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom_start=10
)

geo_json = GeoJson(
    data=geojson,
    style_function=lambda x: {
        'opacity': 1, 'dashArray': '1', 'fillOpacity': 0, 'weight': 1
    },
)
geo_json.add_to(m)
m

(-94.6911621, 37.0332547, -94.402771, 37.1077651)


### Get File info

In [8]:
from rio_tiler.io import COGReader
with COGReader(geojson["features"][0]["assets"]["COG"]["href"]) as cog:
    info = cog.info().dict(exclude_none=True)
    print(info)
    minzoom = info["minzoom"]
    maxzoom = info["maxzoom"]

{'bounds': BoundingBox(left=-94.68841552734375, bottom=37.03325468997235, right=-94.65545654296875, top=37.05956083025124), 'center': (-94.67193603515625, 37.0464077601118, 13), 'minzoom': 13, 'maxzoom': 18, 'band_metadata': [('1', {}), ('2', {}), ('3', {})], 'band_descriptions': [('1', ''), ('2', ''), ('3', '')], 'dtype': 'uint8', 'nodata_type': 'Nodata', 'colorinterp': ['red', 'green', 'blue'], 'driver': 'GTiff', 'overviews': [2, 4, 8, 16], 'height': 6144, 'count': 3, 'nodata_value': 0.0, 'width': 6144}


In [9]:
# create Mosaic
body = {
    "collections": ["joplin"],
    "fields": {
        "includes": ["assets", "id", "bbox"],
    },
    "limit": 20,
}

r = requests.post(
    f"{endpoint}/create",
    json=body,
    params={"minzoom": minzoom, "maxzoom": maxzoom, "assets": "COG"}
).json()
print(r)

{'tilejson': '2.2.0', 'name': '74146fa314ef113252035ccc0bf4c20e', 'description': None, 'version': '1.0.0', 'attribution': None, 'template': None, 'legend': None, 'scheme': 'xyz', 'tiles': ['http://0.0.0.0:8081/tiles/74146fa314ef113252035ccc0bf4c20e/WebMercatorQuad/{z}/{x}/{y}@1x?assets=COG'], 'grids': None, 'data': None, 'minzoom': 13, 'maxzoom': 18, 'bounds': [-180.0, -90.0, 180.0, 90.0], 'center': [0.0, 0.0, 13]}


In [10]:
m = Map(
    location=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom_start=13
)

geo_json = GeoJson(
    data=geojson,
    style_function=lambda x: {
        'opacity': 1, 'dashArray': '1', 'fillOpacity': 0, 'weight': 1
    },
)
geo_json.add_to(m)

aod_layer = TileLayer(
    tiles=r["tiles"][0],
    attr="Mosaic"
)
aod_layer.add_to(m)
m